# Spatial analysis of simulation results

In [184]:
import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import ipywidgets as widgets
import os
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import cm

In [20]:
# Plotting parameters

XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 120

In [8]:
# Config

gcms = ['canesm2_RCP85', 'ccsm4_RCP85', 'giss_e2_h_RCP85', 'noresm1_m_RCP85', 'PRISM']
sim_start = pd.to_datetime('01-01-1984')
sim_end = pd.to_datetime('12-31-2020')

In [9]:
# Import results files

scenarios = ['historical']

dailies = []
annuals = []

# Import daily and annual results
# Results in nested lists ([x][y], for x management scenarios and y GCMs)
for scenario in scenarios:
    dailies_scenario = []
    annuals_scenario = []
    scenario_dir = config.velma_data.parents[1] / 'results' / scenario
    dirs = os.listdir(scenario_dir)
    for gcm in gcms:
        results_dir = scenario_dir / 'ellsworth_{}_{}_{}_{}'.format(scenario,
                                                                    sim_start.year % 100,
                                                                    sim_end.year % 100,
                                                                    gcm)
        results_dir = scenario_dir / directory

        daily_results = pd.read_csv(results_dir / 'DailyResults.csv')

        # Format datetime
        jday_pad = daily_results['Day'].apply(lambda x: str(x).zfill(3))
        str_year = daily_results['Year'].apply(lambda x: str(x))
        rng = pd.to_datetime((str_year + jday_pad), format='%Y%j')
        daily_results.index = rng
        dailies_scenario.append(daily_results)

    dailies.append(dailies_scenario)

## Biomass carbon

In [187]:
# Trim the image to only the delineated watershed simulated through VELMA
# Can export a delineated DEM from JPDEM
del_dem = np.loadtxt(config.dem_velma.parents[0] / 'delineated_dem.asc', skiprows=6)
watershed_mask = (del_dem==-9999)

biomass_c_dir = results_dir / 'spatial_writer' / 'biomass_c'

asciis = []
for file in os.listdir(biomass_c_dir):
    asc = np.loadtxt(biomass_c_dir / file, skiprows=6)
    asc[watershed_mask] = np.nan
    asciis.append(asc)

years = []
for file in os.listdir(biomass_c_dir):
    year = Path(file).stem.split('_')[-2]
    years.append(year)
years = [int(x) for x in dates]

In [196]:
binary_cmap = cm.get_cmap('binary', 2)
gray = np.array([0.31, 0.31, 0.31, 1])
binary_cmap_colors = binary_cmap(np.linspace(0, 1, 2))
binary_cmap_colors[:1, :] = gray
binary_cmap_gray = colors.ListedColormap(binary_cmap_colors)

@widgets.interact(year=(1984, 2020))
def f(year=min(years)):
    ind = dates.index(year)
    fig, ax = plt.subplots(figsize=(7, 8))
    im = ax.imshow(asciis[ind], cmap='YlGn')
    mask = np.isnan(asciis[ind]) * 1.0
    mask[mask == 0] = np.nan
    ax.imshow(mask, cmap=binary_cmap_gray)
    cbar = fig.colorbar(im, ax=ax)
    cbar.ax.set_ylabel(r'$gC/m^2$', rotation=270)

interactive(children=(IntSlider(value=1984, description='year', max=2020, min=1984), Output()), _dom_classes=(…